In [ ]:
import os
os.chdir('/mnt/c/Users/ralvin/OneDrive - Reliant Health Partners/Documents/RHP_dev_RA/Automate_skyvia')
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime
from helper_functions_v2 import create_df, updated_data_pull, upsert, delete_record, insert_records, salesforce_connection, read_sftp_data
import configparser
import json
import paramiko
import io

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


##### retrieve source data
config = configparser.ConfigParser()
config.read('config.ini')
sftp_config = config['sftp']
mapping_config = config['mappings']
host = sftp_config.get('host')
port = sftp_config.getint('port', fallback=22) 
username = sftp_config.get('user')
password = sftp_config.get('password')
remote_path = sftp_config.get('remote_path')
json_path = mapping_config.get('json_file_path')


Source_data =  read_sftp_data(remote_path, host, port, username, password)
print(f'Source data shape: {Source_data.shape}')


## get data mappings 
with open(json_path, 'r') as f:
    data = json.load(f)

mappings = {k:v for k,v in data.items()}
Claims_map = mappings['Claim_Object_Map']
Account_map = mappings['Account_Object_Map']
Povider_map = mappings['Provider_Object_Map']



## sf connection
sf = salesforce_connection(sandbox=True)
sf_prod = salesforce_connection(sandbox=False)


## data pull
# sf_objects = ["Claims__c", "Line_Items__c", "Provider_TIN__c", 'Jurisdiction__c', 'Groups_Clients__c', 'DRG__c', 
#                  'HCPCS_CPT_Code__c', 'Account', 'Clients__c', 'Provider_Specialty__c']
# my_sf_objects = updated_data_pull(sf_objects, sf)
# for key in my_sf_objects:
#     print(key)
#     print(my_sf_objects[key].shape)




#### logging is done on a file level... ideally we archive file into review dir and have info on all errors with respect to file 


In [ ]:
claims_sf = updated_data_pull(['Claims__c'], sf)
print(claims_sf.shape)

In [9]:
Line_items_sf = updated_data_pull(['Line_Items__c'], sf)
print(Line_items_sf.shape)

pulled Line_Items__c
(22841, 49)


In [35]:

def data_pull(object_name, id_list, sf):
    
    sf_object = getattr(sf, object_name)

    # Retrieve and print field names
    metadata = sf_object.describe()
    field_names = [field['name'] for field in metadata['fields']]

    # Construct the SOQL query with ID filtering
    id_filter = "', '".join(id_list)
    query = f"SELECT {', '.join(field_names)} FROM {object_name} WHERE Id IN ('{id_filter}')"
    
    # Retrieve records
    records = sf.query(query)
    all_records = records['records']
    

    # Handle pagination
    while not records['done']:
        records = sf.query_more(records['nextRecordsUrl'], True)
        all_records.extend(records['records'])

    # Convert to DataFrame
    df = pd.DataFrame(all_records)

    # Remove Salesforce metadata keys if present
    df = df.drop(columns=['attributes'], errors='ignore')

    print('pulled data slice!')

    return df



def create_df(source_dict, Source_data, sf, id_list = None):
    new_columns = [col for mapped_cols in source_dict['MAPPINGS'].values() for col in mapped_cols]
    new_df = pd.DataFrame(columns=new_columns)
    keys = source_dict['KEYS']
    try:
        Table_convert_cols = list(source_dict['TABLE_CONVERT'].keys())
    except:
        Table_convert_cols = []
    for source_col, target_cols in source_dict['MAPPINGS'].items():
        if source_col in Table_convert_cols:
            map_table = source_dict['TABLE_CONVERT'][source_col]
            try:
                string_ids = [
                        str(int(float(id))) if pd.notna(id) and id != 'nan' else np.nan
                        for id in Source_data[source_col]                               #### handeling DRG col
                    ]
            except:
                string_ids = [str(id) for id in Source_data[source_col]]
            Source_data[source_col] = string_ids #### col update to match same type in foreign table
            foreign_table = list(map_table.keys())[0]
            print(f'pulling updated {foreign_table} table')
        
            try:
                sf_object = data_pull(foreign_table, id_list, sf)  #### passing in idlist to prevent pulling entire object
            except Exception as e:
                sf_object = None  # Ensure sf_object is set to None if there's an exception


            if sf_object is None:
                sf_object = updated_data_pull([foreign_table], sf) #### need logic to pull only the records I need rather than the entire table
                


            merged = Source_data.merge(sf_object.loc[:, map_table[foreign_table]], left_on=source_col, right_on=map_table[foreign_table][0], how='left')
       

            for i, target_col in enumerate(target_cols):
                foreign_col = map_table[foreign_table][i]
                new_df[target_col] = merged[foreign_col]
            


        if source_col not in Table_convert_cols:
            if source_col in Source_data.columns:
                for target_col in target_cols:
                    new_df[target_col] = Source_data[source_col]
        



    return new_df, keys

In [ ]:
##### populate claims obj on sf

map_claims_table, keys = create_df(Claims_map, Source_data, sf)
new_ids, errors = insert_records(map_claims_table.drop_duplicates(), keys, sf) ###insert new claims records


pulling updated Provider_TIN__c table
pulled Provider_TIN__c
pulling updated Groups_Clients__c table
pulled Groups_Clients__c
pulling updated Jurisdiction__c table
pulled Jurisdiction__c
pulling updated DRG__c table
pulled DRG__c
pulling updated Provider_Specialty__c table
pulled Provider_Specialty__c
Processing record 0: {'Claim_ID__c': 'A3038B5E4B6C914tksft', 'Provider_TIN__c': 'a0FVF000001zh5t2AA', 'Group_Client__c': 'a0C8a00000r3GJ6EAM', 'Claim_Number__c': '431817310700011', 'Jurisdiction__c': 'a024W00000ICbumQAD', 'Diag_Code__c': 'M25562', 'Diag_Code_2nd__c': nan, 'Diag_Code_3rd__c': nan, 'Diag_Code_4th__c': nan, 'Provider__c': 'KAISER FOUNDATION HOSPITALS ', 'Patient__c': 'WATKINS MONIC  ', 'Patient_ID__c': '569713835', 'Date_of_Birth__c': '08/12/1982', 'Provider_Zip__c': 900749998, 'DRG__c': nan, 'QPA__c': 'N', 'NPI__c': nan, 'POS__c': 13, 'Provider_Specialty__c': 'a008a000015yXODAA2', 'Service_Type__c': 'OUTPATIENT'}
Processing record 1: {'Claim_ID__c': 'BE9124A610A9998tksft', 


KeyboardInterrupt



In [33]:
codes_sf = updated_data_pull(['HCPCS_CPT_Code__c'], sf)
codes_sf.columns

pulled HCPCS_CPT_Code__c


Index(['attributes', 'Id', 'OwnerId', 'IsDeleted', 'Name', 'CreatedDate',
       'CreatedById', 'LastModifiedDate', 'LastModifiedById', 'SystemModstamp',
       'LastActivityDate', 'LastViewedDate', 'LastReferencedDate',
       'Medciare_Physician__c', 'Avg_Pro_FMP__c', 'Avg_Fac_FMP__c',
       'NJ_QPA__c', 'NJ_Medicare_Phy__c', 'APC__c', 'Medicare_Oupt_Rate__c',
       'Short_Desc__c', 'Cigna_Non_Facility__c', 'Cigna_Facility__c',
       'Cigna_Mod_26__c', 'Cigna_Mod_TC__c', 'Cigna_Mod_QW__c',
       'Medicare_RR__c', 'Mult_Proc__c', 'Medicare_ASC__c', 'BHSG_Pro_Fee__c',
       'TN_QPA__c', 'MO_QPA__c', 'NJ_Outpt_QPA__c', 'TN_Outpt_QPA__c',
       'MO_Outpt_QPA__c', 'Anes_Base_Units__c', 'Anes_Conversion_Factor__c'],
      dtype='object')

In [31]:
merged = Source_data.merge(codes_sf.loc[:,['Name', 'Id']], left_on='HCPCS/CPT Code', right_on='Name', how='left')

merged.head(20)


,Claim ID,Claim Number,Line ID,TIN,Provider,Address,City,State,Zip,Patient ID,Patient,Date of Birth,Patient Group/Policy Number,DRG,Diag Code 1st,Diag Code 2nd,Diag Code 3rd,Diag Code 4th,Begin DOS,End DOS,Revenue Code,HCPCS/CPT Code,Mod,Mod (2nd),Units,Billed Amount,MAR,IsQPA,Billing Provider Taxonomy,JurisdictionState,4LCodes,NPI,PlaceofService,AnesthesiaUnits,Billtype,BillProvAddress1,BillProvAddress2,BillProvCity,BillProvState,BillProvZip,DRGCode,ClientName,Name,Id
0,A3038B5E4B6C914tksft,431817310700011,1,941105628,KAISER FOUNDATION HOSPITALS,FILE 54602,LOS ANGELES,CA,900749998,569713835,WATKINS MONIC,08/12/1982,NNMCMC,NaN,M25562,NaN,NaN,NaN,09/26/2024,09/26/2024,320,73564,NaN,NaN,1,987.00,156.72,N,282N00000X,CA,NaN,NaN,13,,13,FILE 54602,,LOS ANGELES,CA,900749998,,NovaNet - MCMC - RHP,73564,a0b4W00000y1IFrQAM
1,BE9124A610A9998tksft,CLU20241115325204,1,351166081,JOHNSON MEMORIAL HOSPITAL,1125 W JEFFERSON STREET,FRANKLIN,IN,461312140,QWCR14517,FISHER TERRI,01/11/1974,100600,NaN,M7672,M722,M7732,NaN,10/17/2024,10/17/2024,510,G0463,NaN,NaN,1,65.00,65.00,N,282N00000X,IN,NaN,1.578930e+09,13,,13,1125 W JEFFERSON STREET,,FRANKLIN,IN,461312140,,Lucent - RHP,G0463,a0b4W00000y1JRaQAM
2,317D671330C4442tksft,U424319281216,1,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,278,C1889,NaN,NaN,1,3.00,3.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN
3,317D671330C4442tksft,U424319281216,2,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,300,81025,NaN,NaN,1,54.00,54.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,81025,a0b4W00000y1IqZQAU
4,317D671330C4442tksft,U424319281216,3,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88305,59,NaN,4,1000.00,1000.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,88305,a0b4W00000y1INfQAM
5,317D671330C4442tksft,U424319281216,4,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88342,NaN,NaN,1,1159.00,1159.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,88342,a0b4W00000y1INwQAM
6,317D671330C4442tksft,U424319281216,5,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,370,nan,NaN,NaN,3,1227.00,1227.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN
7,317D671330C4442tksft,U424319281216,6,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,636,J2704,NaN,NaN,70,155.00,155.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,J2704,a0b4W00000y1JZ9QAM
8,317D671330C4442tksft,U424319281216,7,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,710,nan,NaN,NaN,3,48.00,48.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN
9,317D671330C4442tksft,U424319281216,8,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,750,43239,NaN,NaN,1,3326.00,3326.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,43239,a0b4W00000y1HcPQAU


In [27]:
Source_data.head()

,Claim ID,Claim Number,Line ID,TIN,Provider,Address,City,State,Zip,Patient ID,Patient,Date of Birth,Patient Group/Policy Number,DRG,Diag Code 1st,Diag Code 2nd,Diag Code 3rd,Diag Code 4th,Begin DOS,End DOS,Revenue Code,HCPCS/CPT Code,Mod,Mod (2nd),Units,Billed Amount,MAR,IsQPA,Billing Provider Taxonomy,JurisdictionState,4LCodes,NPI,PlaceofService,AnesthesiaUnits,Billtype,BillProvAddress1,BillProvAddress2,BillProvCity,BillProvState,BillProvZip,DRGCode,ClientName
0,A3038B5E4B6C914tksft,431817310700011,1,941105628,KAISER FOUNDATION HOSPITALS,FILE 54602,LOS ANGELES,CA,900749998,569713835,WATKINS MONIC,08/12/1982,NNMCMC,NaN,M25562,NaN,NaN,NaN,09/26/2024,09/26/2024,320,73564,NaN,NaN,1,987.0,156.72,N,282N00000X,CA,NaN,NaN,13,,13,FILE 54602,,LOS ANGELES,CA,900749998,,NovaNet - MCMC - RHP
1,BE9124A610A9998tksft,CLU20241115325204,1,351166081,JOHNSON MEMORIAL HOSPITAL,1125 W JEFFERSON STREET,FRANKLIN,IN,461312140,QWCR14517,FISHER TERRI,01/11/1974,100600,NaN,M7672,M722,M7732,NaN,10/17/2024,10/17/2024,510,G0463,NaN,NaN,1,65.0,65.00,N,282N00000X,IN,NaN,1.578930e+09,13,,13,1125 W JEFFERSON STREET,,FRANKLIN,IN,461312140,,Lucent - RHP
2,317D671330C4442tksft,U424319281216,1,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,278,C1889,NaN,NaN,1,3.0,3.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
3,317D671330C4442tksft,U424319281216,2,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,300,81025,NaN,NaN,1,54.0,54.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
4,317D671330C4442tksft,U424319281216,3,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,03/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88305,59,NaN,4,1000.0,1000.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP


In [11]:
map_account_table, _ = create_df(Account_map, Source_data, sf)
map_provider_table, keys = create_df(Povider_map, Source_data, sf)
###upsert account and provider records
new_ids, errors = upsert(map_provider_table.drop_duplicates(), keys, sf)




In [12]:
map_claims_table, keys = create_df(Claims_map, Source_data, sf)
new_ids, errors = insert_records(map_claims_table.drop_duplicates(), keys, sf) ###insert new claims records





pulling updated Provider_TIN__c table
pulled Provider_TIN__c
pulling updated Groups_Clients__c table
pulled Groups_Clients__c
pulling updated Jurisdiction__c table
pulled Jurisdiction__c
pulling updated DRG__c table
pulled DRG__c
pulling updated Provider_Specialty__c table
pulled Provider_Specialty__c
Processing record 0: {'Claim_ID__c': 'A3038B5E4B6C914tksft', 'Provider_TIN__c': 'a0FVF000001zh5t2AA', 'Group_Client__c': 'a0C8a00000r3GJ6EAM', 'Claim_Number__c': '431817310700011', 'Jurisdiction__c': 'a024W00000ICbumQAD', 'Diag_Code__c': 'M25562', 'Diag_Code_2nd__c': nan, 'Diag_Code_3rd__c': nan, 'Diag_Code_4th__c': nan, 'Provider__c': 'KAISER FOUNDATION HOSPITALS ', 'Patient__c': 'WATKINS MONIC  ', 'Patient_ID__c': '569713835', 'Date_of_Birth__c': '08/12/1982', 'Provider_Zip__c': 900749998, 'DRG__c': nan, 'QPA__c': 'N', 'NPI__c': nan, 'POS__c': 13, 'Provider_Specialty__c': 'a008a000015yXODAA2', 'Service_Type__c': 'OUTPATIENT'}
Processing record 1: {'Claim_ID__c': 'BE9124A610A9998tksft', 

In [36]:
#### create lines table:
lines_map = mappings['Line_Item_Object_Map']
map_lines_table, keys = create_df(lines_map, Source_data, sf, new_ids)
map_lines_table.shape


pulling updated Claims__c table
pulled data slice!
pulling updated HCPCS_CPT_Code__c table
pulled data slice!


KeyError: "None of [Index(['Name', 'Id'], dtype='object')] are in the [columns]"

In [5]:
source_cases = Source_data.drop_duplicates(subset=['Claim ID'])[['Claim ID']]
ids = list(source_cases.merge(updated_data_pull(['Claims__c'], sf), left_on='Claim ID', right_on='Claim_ID__c').loc[:, ['Claim ID', 'Claim_ID__c', 'Id']]['Id'])
len(ids)

pulled Claims__c


103

In [6]:
################# delete records:
keys = {'Claims__c':'Claim_ID__c'}
delete_record(ids, keys, sf)

Object: Claims__c, Records to process: 103
All records deleted successfully.


In [2]:
# new_claim_ids, error_logs = insert_records(map_claims_table.drop_duplicates(), keys, sf)

In [3]:
groups_clients_sf = my_sf_objects['Groups_Clients__c']
groups_clients_sf.shape

(1703, 59)

In [5]:
map_claims_table[map_claims_table['Group_Client__c'].isna()].drop_duplicates()

,Claim_ID__c,Provider_TIN__c,Group_Client__c,Claim_Number__c,Jurisdiction__c,Diag_Code__c,Diag_Code_2nd__c,Diag_Code_3rd__c,Diag_Code_4th__c,Provider__c,Patient__c,Patient_ID__c,Date_of_Birth__c,Provider_Zip__c,DRG__c,QPA__c,NPI__c,POS__c,Provider_Specialty__c,Service_Type__c
116,1E6E757A964D548tksft,a0F4W00000W1KH5UAN,NaN,18A91B0008179A1,a024W00000HYAjJQAX,C20,C7951,R6889,NaN,METHODIST HOSPITALSINC.,ROODZANT JAMES,880263595,12/22/1971,464107035,NaN,N,1.518035e+09,13,NaN,NaN
202,B17551A5CD4B233tksft,a0F4W00000W1LEEUA3,NaN,18A91B0008179A8,a024W00000HYAjRQAX,Z5181,Z79899,NaN,NaN,FREEMAN NEOSHO HOSPITAL,WILSON ROBERT L,880440969,03/12/1960,648501705,NaN,N,1.154990e+09,85,NaN,NaN
214,24BF09BD7288E97tksft,a0FVF0000020PQH2A2,NaN,06A11B00081784F,a024W00000HYAjkQAH,M25532,X500XXA,NaN,NaN,PFLUGERVILLE EMERGENCY CENTER LLC,INABINETT COLE,750076498,05/27/2011,786605965,NaN,Y,1.154762e+09,13,NaN,NaN
272,0D8755A2AD0EBC0tksft,a0FVF0000020Onc2AE,NaN,P24319I2000001,a024W00000HYAjRQAX,Z01818,N871,NaN,NaN,CURATORS OF THE UNIVERSITY,ARNOLD ASHLEY,489984314,05/19/1988,652120001,NaN,Y,NaN,22,NaN,NaN
374,0B46B602A7B3B67tksft,a0FVF0000020PbZ2AU,NaN,21040838Accredited,a024W00000HYAj9QAH,M5416,NaN,NaN,NaN,SURGCENTER NORTHERN PHOENIX,PEN KEVIN,XXXXX1120,11/22/1981,850859998,NaN,N,NaN,83,a008a000015yXNEAA2,SURGERY CENTER
431,802EF0AD00A30E7tksft,a0FVF0000020Pen2AE,NaN,18A91B0008179EE,a024W00000HYAjoQAH,M2352,NaN,NaN,NaN,SEATTLE CHILDRENS HOSPITAL,JOHNSTON HARPER A,880402012,09/16/2009,980043829,NaN,N,1.316174e+09,13,NaN,NaN


In [18]:
for i in sorted(list(groups_clients_sf['Group_Number__c'])):
    if '' in i:
        print(i)

330803325RELAccidentFund
